In [1]:
import pandas as pd
from datetime import date, datetime, timedelta
import time
import numpy as np

import FinanceDataReader as fdr
from pykrx import stock

import requests
import bs4
# from selenium import webdriver
# from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
# from selenium.webdriver.chrome.options import Options
import json
import xml.etree.ElementTree as ET

from urllib import request
from urllib.request import urlopen # HTTP 요청처리
from zipfile import ZipFile
import sys
import os                          
import xmltodict                   # xml을 dict로 파싱
from pathlib import Path           # file 존재유무 체크 유틸
import shutil
from io import BytesIO

In [2]:
nowDate = datetime.today()

startDate = (nowDate - timedelta(7)).strftime("%Y%m%d")
endDate = (nowDate - timedelta(1)).strftime("%Y%m%d")

testStartDate = (nowDate - timedelta(97)).strftime("%Y%m%d")
testEndDate = (nowDate - timedelta(91)).strftime("%Y%m%d")
print(startDate, endDate, testStartDate, testEndDate)

20200624 20200630 20200326 20200401


In [3]:
# dart corp_code 수집
def bind_params(params : dict):
    url_params = []
    for key in params:
        url_params.append(key + '=' + params[key])
    return url_params

def has_corpfile():
    dirpath = os.getcwd()

    if Path(dirpath + '/CORPCODE.xml').is_file():
        return True
    else:
        return False

def get_corp_xml(url, params):

    if has_corpfile():
        dirpath = os.getcwd()
        return open(dirpath + '/CORPCODE.xml', 'r').read()
    else:
        url = url + '&'.join(bind_params(params))
        resp = urlopen(url)
        zipfile = ZipFile(BytesIO(resp.read()))
        print(zipfile.namelist())
        for name in zipfile.namelist():
            z = zipfile.open(name)  
            with open(name, 'w') as codefile:
                for l in z.readlines():
                    codefile.write(l.decode()) 
        dirpath = os.getcwd()
        return open(dirpath + '/CORPCODE.xml', 'r').read()

### 종목코드(Ticker) 및 기타 정보 수집

In [4]:
def getTickerDF(targetDate):
    tickersDF = pd.read_csv("./tickersDF.csv", encoding="ms949", dtype = {"Ticker":str}, index_col=False)
    tickersDF = tickersDF[tickersDF.Sector.notnull()].reset_index(drop=True)
    dropList = []
    tmpIndex = 0
    for eachListingDate in tickersDF.ListingDate:
        if (datetime.strptime(eachListingDate, "%Y-%m-%d") + timedelta(targetDate) > nowDate):
            dropList.append(tmpIndex)
        else :
            pass
        tmpIndex = tmpIndex + 1
    tickersDF = tickersDF.drop(index = dropList).reset_index(drop=True)
    return tickersDF

In [5]:
# FinanceDataReader : KRX
tickersDF = fdr.StockListing('KRX')

In [6]:
tickersDF = tickersDF.rename(columns={"Symbol":"Ticker"})

In [7]:
tickersDF.to_csv("./tickersDF.csv", encoding="ms949", index=False)

In [8]:
# 보통주만 발라내기
tickersDF = tickersDF[tickersDF.Sector.notnull()]

In [9]:
tickersDF

,Ticker,Market,Name,Sector,Industry,ListingDate,SettleMonth,Representative,HomePage,Region
0,060310,KOSDAQ,3S,특수 목적용 기계 제조업,반도체 웨이퍼 캐리어,2002-04-23,03월,"박종익, 김세완 (각자 대표이사)",http://www.3sref.com,서울특별시
1,095570,KOSPI,AJ네트웍스,산업용 기계 및 장비 임대업,"렌탈(파렛트, OA장비, 건설장비)",2015-08-21,12월,반채운 윤규선 이현우,http://www.ajnet.co.kr,서울특별시
2,006840,KOSPI,AK홀딩스,기타 금융업,지주사업,1999-08-11,12월,"채형석, 이석주(각자 대표이사)",http://www.aekyunggroup.co.kr,서울특별시
3,054620,KOSDAQ,APS홀딩스,기타 금융업,인터넷 트래픽 솔루션,2001-12-04,12월,정기로,http://www.apsholdings.co.kr,경기도
4,265520,KOSDAQ,AP시스템,특수 목적용 기계 제조업,디스플레이 제조 장비,2017-04-07,12월,김영주,http://www.apsystems.co.kr,경기도
...,...,...,...,...,...,...,...,...,...,...
2546,189980,KOSDAQ,흥국에프엔비,비알코올음료 및 얼음 제조업,"과일음료(에이드베이스, 스무디, 착즙쥬스 등)",2015-08-07,12월,"박철범, 오길영",http://www.hyungkuk.com,서울특별시
2547,000540,KOSPI,흥국화재,보험업,손해보험,1974-12-05,12월,권중원,http://www.insurance.co.kr,서울특별시
2550,003280,KOSPI,흥아해운,해상 운송업,"해상운송(일본항로,동남아항로)",1976-06-29,12월,이환구,http://www.heung-a.co.kr,서울특별시
2551,037440,KOSDAQ,희림,"건축기술, 엔지니어링 및 관련 기술 서비스업",설계 및 감리용역,2000-02-03,12월,"정영균, 이목운, 허철호, 염두성 (각자대표)",http://www.heerim.com,서울특별시


### 고유번호(CorpCode) 수집 

In [10]:
url = 'https://opendart.fss.or.kr/api/corpCode.xml?'
params = {'crtfc_key':'b7ea653fdd00a66f74a44b32a0dca161a128629c'}

In [11]:
corp_xml = get_corp_xml(url, params)
corp_dict = xmltodict.parse(corp_xml)
corp_list = corp_dict['result']['list']
corp_list_has_stockcode = [x for x in corp_list if x['stock_code'] is not None]

In [12]:
corp_DF = pd.DataFrame(corp_list_has_stockcode)

In [13]:
corp_DF = corp_DF.drop(["modify_date"], axis="columns")

In [14]:
corp_DF = corp_DF.rename(columns={"corp_code":"CorpCode","corp_name":"CorpName","stock_code":"Ticker"})

In [15]:
corp_DF

,CorpCode,CorpName,Ticker
0,00260985,한빛네트,036720
1,00264529,엔플렉스,040130
2,00358545,동서정보기술,055000
3,00231567,애드모바일,032600
4,00247939,씨모스,037600
...,...,...,...
3197,00545929,제넥신,095700
3198,00937324,한국타이어앤테크놀로지,161390
3199,00391197,메디프론,065650
3200,00995993,캔서롭,180400


### 종목코드DF와 고유번호DF와 merge

In [16]:
infoDF = pd.merge(tickersDF, corp_DF, how="inner")

In [17]:
infoDF

,Ticker,Market,Name,Sector,Industry,ListingDate,SettleMonth,Representative,HomePage,Region,CorpCode,CorpName
0,060310,KOSDAQ,3S,특수 목적용 기계 제조업,반도체 웨이퍼 캐리어,2002-04-23,03월,"박종익, 김세완 (각자 대표이사)",http://www.3sref.com,서울특별시,00378363,3S
1,095570,KOSPI,AJ네트웍스,산업용 기계 및 장비 임대업,"렌탈(파렛트, OA장비, 건설장비)",2015-08-21,12월,반채운 윤규선 이현우,http://www.ajnet.co.kr,서울특별시,00365387,AJ네트웍스
2,006840,KOSPI,AK홀딩스,기타 금융업,지주사업,1999-08-11,12월,"채형석, 이석주(각자 대표이사)",http://www.aekyunggroup.co.kr,서울특별시,00125080,AK홀딩스
3,054620,KOSDAQ,APS홀딩스,기타 금융업,인터넷 트래픽 솔루션,2001-12-04,12월,정기로,http://www.apsholdings.co.kr,경기도,00296078,APS홀딩스
4,265520,KOSDAQ,AP시스템,특수 목적용 기계 제조업,디스플레이 제조 장비,2017-04-07,12월,김영주,http://www.apsystems.co.kr,경기도,01203808,AP시스템
...,...,...,...,...,...,...,...,...,...,...,...,...
2346,189980,KOSDAQ,흥국에프엔비,비알코올음료 및 얼음 제조업,"과일음료(에이드베이스, 스무디, 착즙쥬스 등)",2015-08-07,12월,"박철범, 오길영",http://www.hyungkuk.com,서울특별시,00916516,흥국에프엔비
2347,000540,KOSPI,흥국화재,보험업,손해보험,1974-12-05,12월,권중원,http://www.insurance.co.kr,서울특별시,00103176,흥국화재
2348,003280,KOSPI,흥아해운,해상 운송업,"해상운송(일본항로,동남아항로)",1976-06-29,12월,이환구,http://www.heung-a.co.kr,서울특별시,00167208,흥아해운
2349,037440,KOSDAQ,희림,"건축기술, 엔지니어링 및 관련 기술 서비스업",설계 및 감리용역,2000-02-03,12월,"정영균, 이목운, 허철호, 염두성 (각자대표)",http://www.heerim.com,서울특별시,00236863,희림


### 모든 기업의 재무재표 JSON으로 가져오기

In [18]:
headers = {'User-Agent':'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36'}

In [19]:
apiKey = "b7ea653fdd00a66f74a44b32a0dca161a128629c"
apiKey2 = "ab741ce7fa417a68b34d6d5cbf4e5bcbe247f0c8"

In [20]:
# # 사업보고서 : 결산 후 90일 이내. 4월 1일부터 사업보고서 조회 가능 
# # 분기보고서 : 분기 결산 후 45일 이내. 5월 16일 1분기보고서 조회 가능 , 11월 16일부터 3분기보고서 조회 가능
# # 반기보고서 : 반기 결산 후 45일 이내 8월 16일부터 반기보고서 조회 가능

# # today : 20200627처럼 8자리 문자열 
# def calcTargetYearAndQuater():
#     today = datetime.today()
#     todayStr = today.strftime("%Y%m%d") 
#     nowYearStr = todayStr[:4]
#     quater1Str = nowYearStr + "0516"
#     quater2Str = nowYearStr + "0816"
#     quater3Str = nowYearStr + "1116"
#     quater1 = datetime.strptime(quater1Str, "%Y%m%d")
#     quater2 = datetime.strptime(quater2Str, "%Y%m%d")
#     quater3 = datetime.strptime(quater3Str, "%Y%m%d")
#     if (today < quater1):
#         targetYearAndQuater = [str(int(nowYearStr)-1), "4Q"]
#     elif (quater1 < today < quater2):
#         targetYearAndQuater = [nowYearStr, "1Q"]
#     elif (quater2 < today < quater3):
#         targetYearAndQuater = [nowYearStr, "2Q"]
#     elif (quater3 < today):
#         targetYearAndQuater = [nowYearStr, "3Q"]
    
#     return targetYearAndQuater

In [21]:
# 사업보고서 : 결산 후 90일 이내. 4월 1일부터 사업보고서 조회 가능 
# 분기보고서 : 분기 결산 후 45일 이내. 5월 16일 1분기보고서 조회 가능 , 11월 16일부터 3분기보고서 조회 가능
# 반기보고서 : 반기 결산 후 45일 이내 8월 16일부터 반기보고서 조회 가능

# today : 20200627처럼 8자리 문자열 
def calcTargetYearAndQuater():
    today = datetime.today()
    todayStr = today.strftime("%Y%m%d") 
    nowYearStr = todayStr[:4]
    quater1Str = nowYearStr + "0516"
    quater2Str = nowYearStr + "0816"
    quater3Str = nowYearStr + "1116"
    quater1 = datetime.strptime(quater1Str, "%Y%m%d")
    quater2 = datetime.strptime(quater2Str, "%Y%m%d")
    quater3 = datetime.strptime(quater3Str, "%Y%m%d")
    if (today < quater1):
        targetYearAndQuater = [str(int(nowYearStr)-1), "4Q"]
    elif (quater1 <= today < quater2):
        targetYearAndQuater = [nowYearStr, "1Q"]
    elif (quater2 <= today < quater3):
        targetYearAndQuater = [nowYearStr, "2Q"]
    elif (quater3 <= today):
        targetYearAndQuater = [nowYearStr, "3Q"]
    
    return targetYearAndQuater

In [22]:
targetYearAndQuater = calcTargetYearAndQuater()

In [23]:
targetYearAndQuater

['2020', '1Q']

In [24]:
targetYear = targetYearAndQuater[0]
targetQuater = targetYearAndQuater[1]

In [25]:
def createUrlToGetFS(apiKey, corpCode, bsnsYear, quater):
    if (quater == "1Q"):
        reprt_code = "11013" # 1분기보고서
    elif (quater == "2Q"):
        reprt_code = "11012" # 반기보고서
    elif (quater == "3Q"):
        reprt_code = "11014" # 3분기보고서
    elif (quater == "4Q"):
        reprt_code = "11011" # 사업보고서
    url = "https://opendart.fss.or.kr/api/fnlttSinglAcntAll.json?crtfc_key={}&corp_code={}&bsns_year={}&reprt_code={}&fs_div=OFS".format(apiKey, corpCode, bsnsYear, reprt_code)
    return url

In [26]:
def getAllFinancialSheet(apiKey, corpCodeList, targetYear, targetQuater):
    totalJsonList = []
    for eachCorpCode in corpCodeList:
        eachUrl = createUrlToGetFS(apiKey, eachCorpCode, targetYear, targetQuater)
        try:
            eachJson = requests.get(eachUrl, headers = headers).json()
            if (eachJson["status"] == "000"):
                totalJsonList.append(eachJson["list"])
        except:
            try:
                eachJson = requests.get(eachUrl, headers = headers).json()
                time.sleep(0.5)
                if (eachJson["status"] == "000"):
                    totalJsonList.append(eachJson["list"])
            except:
                print(eachUrl + "로부터 정보를 가져오지 못했습니다.")
    
    return totalJsonList

In [27]:
corpCodeList = infoDF.CorpCode

In [28]:
allFsList = getAllFinancialSheet(apiKey, corpCodeList, targetYear, targetQuater)

### 모든 기업의 재무재표로부터 필요한 자료 추출(매출액 등)

In [29]:
def getAmount(allFsList):
    totalList = []
    for eachFsJson in allFsList:
        eachList = []
        eachDf = pd.DataFrame(eachFsJson)
        # 고유번호
        eachCorpCode = eachDf.corp_code[0]
        eachList.append(eachCorpCode)
        
        # 매출액
        try:
            eachSales = eachDf.thstrm_amount[eachDf.sj_nm.str.contains("손익계산서") & 
                                                     (eachDf.account_id == "ifrs-full_Revenue")].reset_index(drop=True)[0]
        except:
            try:
                eachSales = eachDf.thstrm_amount[eachDf.sj_nm.str.contains("손익계산서") & 
                                             eachDf.account_nm.str.contains("영업수익|매출")].reset_index(drop=True)[0]
            except:
                eachSales = 0
        eachList.append(eachSales)
        
        # 당기순이익
        try:
            eachNetIncome = eachDf.thstrm_amount[eachDf.sj_nm.str.contains("손익계산서") & 
                                                 (eachDf.account_id == "ifrs-full_ProfitLoss")].reset_index(drop=True)[0]
        except:
            try:
                eachNetIncome = eachDf.thstrm_amount[eachDf.sj_nm.str.contains("손익계산서") & 
                                                 eachDf.account_nm.str.contains("당기순|분기순")].reset_index(drop=True)[0]
            except:
                eachNetIncome = 0
        eachList.append(eachNetIncome)
        
        # 자산총계
        try:
            eachTotalAssets = eachDf.thstrm_amount[eachDf.sj_nm.str.contains("재무상태표") & 
                                                     (eachDf.account_id == "ifrs-full_Assets")].reset_index(drop=True)[0]
        except:
            try:
                eachTotalAssets = eachDf.thstrm_amount[eachDf.sj_nm.str.contains("재무상태표") & 
                                                   eachDf.account_nm.str.contains("자산총계|부채와자본총계")].reset_index(drop=True)[0]
            except:
                eachTotalAssets = 0
        eachList.append(eachTotalAssets)
        
        # 자본총계
        try:
            eachTotalEquity = eachDf.thstrm_amount[eachDf.sj_nm.str.contains("재무상태표") & 
                                                     (eachDf.account_id == "ifrs-full_Equity")].reset_index(drop=True)[0]
        except:
            try:
                eachTotalEquity = eachDf.thstrm_amount[eachDf.sj_nm.str.contains("재무상태표") & 
                                                   eachDf.account_nm.str.contains("자본총계")].reset_index(drop=True)[0]
            except:
                eachTotalEquity = 0
        eachList.append(eachTotalEquity)
        
        # 영업현금흐름
        try:
            eachOCF = eachDf.thstrm_amount[eachDf.sj_nm.str.contains("현금흐름표") & 
                                                     (eachDf.account_id == "ifrs-full_CashFlowsFromUsedInOperatingActivities")].reset_index(drop=True)[0]
        except:
            try:
                eachOCF = eachDf.thstrm_amount[eachDf.sj_nm.str.contains("현금흐름표") & 
                                                   eachDf.account_nm.str.contains("영업활동")].reset_index(drop=True)[0]
            except:
                eachOCF = 0
        eachList.append(eachOCF)
        
        totalList.append(eachList)

    resultDf = pd.DataFrame(totalList, columns=["CorpCode", "Sales", "NetIncome", "TotalAssets", "TotalEquity", "OCF"])
#     resultDf.to_csv("./amount.csv", encoding="ms949")
    
    return resultDf

In [30]:
amountDf = getAmount(allFsList)

In [31]:
amountDf

,CorpCode,Sales,NetIncome,TotalAssets,TotalEquity,OCF
0,00365387,107018343206,-113230171,1204909858969,294365123538,-27031512284
1,00125080,20537229681,16718068276,637338257823,524836004925,-1276961825
2,00296078,1457331870,-1487461711,240857867934,208480707923,-1153135451
3,01203808,120180783720,7920112710,453125516385,120577381477,8100020626
4,00874803,7254426867,1115308706,107047007337,82516243309,38542687
...,...,...,...,...,...,...
2022,00167031,20012651753,2556346075,74395752982,55652545454,901181715
2023,00916516,9600696524,347405877,87214079361,61959918516,-3067537931
2024,00167208,24243075025,4194012390,374553376752,23809455494,-16713305766
2025,00236863,42592157110,341453948,156836064405,52322113987,-2495009611


In [32]:
amountAndInfoDf = pd.merge(infoDF, amountDf, how="inner")

In [33]:
amountAndInfoDf.head(1)

,Ticker,Market,Name,Sector,Industry,ListingDate,SettleMonth,Representative,HomePage,Region,CorpCode,CorpName,Sales,NetIncome,TotalAssets,TotalEquity,OCF
0,095570,KOSPI,AJ네트웍스,산업용 기계 및 장비 임대업,"렌탈(파렛트, OA장비, 건설장비)",2015-08-21,12월,반채운 윤규선 이현우,http://www.ajnet.co.kr,서울특별시,00365387,AJ네트웍스,107018343206,-113230171,1204909858969,294365123538,-27031512284


### 네이버에서 발행주식수 크롤링

In [34]:
def getNumberOfIssuedShares (tickerList):
    baseUrl = "https://finance.naver.com/item/main.nhn?code="
    numberOfIssuedSharesList = []
    for eachTicker in tickerList:
        eachList = []
        eachUrl = baseUrl + eachTicker
        try:
            response = requests.get(url = eachUrl, headers = headers)
            response.encoding = "ms949"
            html = response.text
            bs = bs4.BeautifulSoup(html, "html.parser")
            numberOfIssuedShares = bs.find(name = "table", attrs = {"summary":"시가총액 정보"}).findAll(name="em")[2].text.replace(",","")
        except:
            try:
                response = requests.get(url = eachUrl, headers = headers)
                response.encoding = "ms949"
                html = response.text
                bs = bs4.BeautifulSoup(html, "html.parser")
                numberOfIssuedShares = bs.find(name = "table", attrs = {"summary":"시가총액 정보"}).findAll(name="em")[1].text.replace(",","")
            except:
                print(eachUrl + " 응답이 없습니다. 정보가 있는 페이지인지 확인이 필요합니다.")
                numberOfIssuedShares = 0
        eachList.append(eachTicker)
        eachList.append(numberOfIssuedShares)
        numberOfIssuedSharesList.append(eachList)
        numberOfIssuedSharesDf = pd.DataFrame(numberOfIssuedSharesList, columns = ["Ticker", "NumOfIssuedShares"])
    
    return numberOfIssuedSharesDf

In [35]:
numOfIssuedSharesDf = getNumberOfIssuedShares(amountAndInfoDf.Ticker)

In [36]:
# numOfIssuedSharesDf.to_csv("./test.csv", encoding="ms949")

In [37]:
amountAndInfoDf = pd.merge(amountAndInfoDf, numOfIssuedSharesDf, how="inner")

In [38]:
amountAndInfoDf.head(1)

,Ticker,Market,Name,Sector,Industry,ListingDate,SettleMonth,Representative,HomePage,Region,CorpCode,CorpName,Sales,NetIncome,TotalAssets,TotalEquity,OCF,NumOfIssuedShares
0,095570,KOSPI,AJ네트웍스,산업용 기계 및 장비 임대업,"렌탈(파렛트, OA장비, 건설장비)",2015-08-21,12월,반채운 윤규선 이현우,http://www.ajnet.co.kr,서울특별시,00365387,AJ네트웍스,107018343206,-113230171,1204909858969,294365123538,-27031512284,46822295


### 종목별 전일 종가 수집

In [39]:
def getAllClose(tickerList, startDate, endDate):
    totalList = []
    for eachTicker in tickerList:
        eachList = []
        eachList.append(eachTicker)
        priceDf = fdr.DataReader(symbol = eachTicker, start = startDate, end = endDate)
        eachClose = priceDf.Close[-1]
        eachList.append(eachClose)
        totalList.append(eachList)
        
    closeDf = pd.DataFrame(totalList, columns=["Ticker", "Close"])
    
    return closeDf

In [40]:
closeDf = getAllClose(amountAndInfoDf.Ticker, startDate, endDate)

In [41]:
closeDf.head(1)

,Ticker,Close
0,095570,2860


In [42]:
amountAndInfoDf = pd.merge(amountAndInfoDf, closeDf, how="inner")

In [43]:
amountAndInfoDf.head(1)

,Ticker,Market,Name,Sector,Industry,ListingDate,SettleMonth,Representative,HomePage,Region,CorpCode,CorpName,Sales,NetIncome,TotalAssets,TotalEquity,OCF,NumOfIssuedShares,Close
0,095570,KOSPI,AJ네트웍스,산업용 기계 및 장비 임대업,"렌탈(파렛트, OA장비, 건설장비)",2015-08-21,12월,반채운 윤규선 이현우,http://www.ajnet.co.kr,서울특별시,00365387,AJ네트웍스,107018343206,-113230171,1204909858969,294365123538,-27031512284,46822295,2860


### 분석 대상 지표 계산 : (-)는 분석 대상에서 제외
##### MarketValue = NumOfIssuedShares * Close
##### ROE = NetIncome / TotalEquity
##### ROA = NetIncome / TotalAssets
##### PER = MarketValue / NetIncome
##### PBR = MarketValue / TotalEquity
##### PCR = MarketValue / OCF
##### PSR = MarketValue / Sales

In [44]:
finalResultDf = amountAndInfoDf.copy()

In [45]:
finalResultDf["MarketValue"] = pd.to_numeric(finalResultDf.NumOfIssuedShares) * pd.to_numeric(finalResultDf.Close)
finalResultDf["ROE"] = pd.to_numeric(finalResultDf.NetIncome) / pd.to_numeric(finalResultDf.TotalEquity)
finalResultDf["ROA"] = pd.to_numeric(finalResultDf.NetIncome) / pd.to_numeric(finalResultDf.TotalAssets)
finalResultDf["PER"] = pd.to_numeric(finalResultDf.MarketValue) / pd.to_numeric(finalResultDf.NetIncome)
finalResultDf["PBR"] = pd.to_numeric(finalResultDf.MarketValue) / pd.to_numeric(finalResultDf.TotalEquity)
finalResultDf["PCR"] = pd.to_numeric(finalResultDf.MarketValue) / pd.to_numeric(finalResultDf.OCF)
finalResultDf["PSR"] = pd.to_numeric(finalResultDf.MarketValue) / pd.to_numeric(finalResultDf.Sales)

### NetIncome 또는 OCF가 (-)인 행 제거

In [46]:
finalResultDf = finalResultDf[(pd.to_numeric(finalResultDf.NetIncome) > 0) & (pd.to_numeric(finalResultDf.OCF) > 0)]

In [47]:
finalResultDf.to_csv("./finalResultDf.csv", encoding="ms949", index=False)

## 테스트용 데이터 수집 (이전 분기)

In [48]:
# 사업보고서 : 결산 후 90일 이내. 4월 1일부터 사업보고서 조회 가능 
# 분기보고서 : 분기 결산 후 45일 이내. 5월 16일 1분기보고서 조회 가능 , 11월 16일부터 3분기보고서 조회 가능
# 반기보고서 : 반기 결산 후 45일 이내 8월 16일부터 반기보고서 조회 가능

# today : 20200627처럼 8자리 문자열 
def calcTestYearAndQuater():
    today = datetime.today() - timedelta(90)
    todayStr = today.strftime("%Y%m%d") 
    nowYearStr = todayStr[:4]
    quater1Str = nowYearStr + "0516"
    quater2Str = nowYearStr + "0816"
    quater3Str = nowYearStr + "1116"
    quater1 = datetime.strptime(quater1Str, "%Y%m%d")
    quater2 = datetime.strptime(quater2Str, "%Y%m%d")
    quater3 = datetime.strptime(quater3Str, "%Y%m%d")
    if (today < quater1):
        targetYearAndQuater = [str(int(nowYearStr)-1), "4Q"]
    elif (quater1 <= today < quater2):
        targetYearAndQuater = [nowYearStr, "1Q"]
    elif (quater2 <= today < quater3):
        targetYearAndQuater = [nowYearStr, "2Q"]
    elif (quater3 <= today):
        targetYearAndQuater = [nowYearStr, "3Q"]
    
    return targetYearAndQuater

In [49]:
testYearAndQuater = calcTestYearAndQuater()

In [50]:
testYearAndQuater

['2019', '4Q']

In [51]:
testYear = testYearAndQuater[0]
testQuater = testYearAndQuater[1]

In [52]:
testAllFsList = getAllFinancialSheet(apiKey, corpCodeList, testYear, testQuater)

In [53]:
testAmountDf = getAmount(testAllFsList)

In [54]:
testAmountAndInfoDf = pd.merge(infoDF, testAmountDf, how="inner")

In [55]:
testAmountAndInfoDf = pd.merge(testAmountAndInfoDf, numOfIssuedSharesDf, how="inner")

In [56]:
testCloseDf = getAllClose(testAmountAndInfoDf.Ticker, testStartDate, testEndDate)

In [57]:
testAmountAndInfoDf = pd.merge(testAmountAndInfoDf, testCloseDf, how="inner")

In [58]:
testFinalResultDf = testAmountAndInfoDf.copy()

In [59]:
testFinalResultDf["MarketValue"] = pd.to_numeric(testFinalResultDf.NumOfIssuedShares) * pd.to_numeric(testFinalResultDf.Close)
testFinalResultDf["ROE"] =  pd.to_numeric(testFinalResultDf.NetIncome) / pd.to_numeric(testFinalResultDf.TotalEquity)
testFinalResultDf["ROA"] = pd.to_numeric(testFinalResultDf.NetIncome) / pd.to_numeric(testFinalResultDf.TotalAssets)
testFinalResultDf["PER"] = pd.to_numeric(testFinalResultDf.MarketValue) / pd.to_numeric(testFinalResultDf.NetIncome)
testFinalResultDf["PBR"] = pd.to_numeric(testFinalResultDf.MarketValue) / pd.to_numeric(testFinalResultDf.TotalEquity)
testFinalResultDf["PCR"] = pd.to_numeric(testFinalResultDf.MarketValue) / pd.to_numeric(testFinalResultDf.OCF)
testFinalResultDf["PSR"] = pd.to_numeric(testFinalResultDf.MarketValue) / pd.to_numeric(testFinalResultDf.Sales)

In [60]:
testFinalResultDf = testFinalResultDf[(pd.to_numeric(testFinalResultDf.NetIncome) > 0) & (pd.to_numeric(testFinalResultDf.OCF) > 0)]

In [61]:
testFinalResultDf.to_csv("./testFinalResultDf.csv", encoding="ms949", index=False)